# First Step, Filtering the Data
In this notebook, we will apply bandpass filtering to the EEG data to retain frequencies between 1 Hz and 45 Hz, which are relevant for most EEG analyses. We will then visualize the power spectral density (PSD) before and after filtering to observe the effects of the filter.

The Artificial Subspace Reconstruction (ASR) is commented out as it is memory intensive and may not be necessary for all datasets.

In [ ]:
import mne
import os
import glob

# Load all resting state files from raw directory
raw_dir = os.path.join('..', '..', 'raw')

# Find all files with "RestingState" in the name
resting_files = glob.glob(os.path.join(raw_dir, "*RestingState*.set"))

for f in resting_files:
    print(f"  - {os.path.basename(f)}")


raw_list = []
for file_path in resting_files:
    raw = mne.io.read_raw_eeglab(file_path, preload=True)
    raw_list.append(raw)

In [ ]:
# high-pass filter at 1 Hz and low-pass filter at 45 Hz
raw = [r.filter(l_freq=1, h_freq=45, fir_design='firwin') for r in raw_list]

In [ ]:
# from asrpy import ASR
# import numpy as np


# for i, r in enumerate(raw):
#     asr = ASR(sfreq=r.info['sfreq'], cutoff=20)
#     asr.fit(r) 

#     raw[i] = asr.transform(r)
# requires alot of computing and memory power, so commented out for now


In [ ]:
# plot to see the effect of filtering
from scipy import signal as sp_signal
import matplotlib.pyplot as plt
import numpy as np

data = raw[0].get_data()

fig, axes = plt.subplots(1, 2, figsize=(18, 8))

# plot 1-45 Hz (valid EEG range)
for i in range(len(raw[0].ch_names)):
    freqs, psd = sp_signal.welch(data[i], fs=raw[0].info['sfreq'], nperseg=2048)
    mask = (freqs >= 1) & (freqs <= 45) & (psd > 0)
    axes[0].semilogy(freqs[mask], psd[mask], alpha=0.3, linewidth=0.5)

axes[0].set_xlabel('Frequency (Hz)', fontsize=14)
axes[0].set_ylabel('Power (V²/Hz)', fontsize=14)
axes[0].set_title('PSD: 0-45 Hz (Valid EEG Range)', fontsize=16, fontweight='bold')
axes[0].set_xlim(0, 45)
axes[0].grid(True, alpha=0.3)

# plot 45-125 Hz (artifact range)
for i in range(len(raw[0].ch_names)):
    freqs, psd = sp_signal.welch(data[i], fs=raw[0].info['sfreq'], nperseg=2048)
    mask = (freqs >= 45) & (freqs <= 125) & (psd > 0)
    axes[1].semilogy(freqs[mask], psd[mask], alpha=0.3, linewidth=0.5)

axes[1].set_xlabel('Frequency (Hz)', fontsize=14)
axes[1].set_ylabel('Power (V²/Hz)', fontsize=14)
axes[1].set_title('PSD: 45-125 Hz (Filtered Artifacts)', fontsize=16, fontweight='bold')
axes[1].set_xlim(45, 125)
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=1e-12, color='red', linestyle='--', linewidth=2, label='Filter cutoff')
axes[1].legend()

plt.tight_layout()
plt.savefig(os.path.join('..', '..', 'figures', 'psd_diagnostic.png'), dpi=150)
plt.show()

In [ ]:
# save cleaned data in fif and numpy formats

processed_dir = os.path.join('..', '..', 'processed')
os.makedirs(processed_dir, exist_ok=True)

# save as fif
for i, r in enumerate(raw):
    output_path_fif = os.path.join(processed_dir, f'resting_state_cleaned_{i}.fif')
    r.save(output_path_fif, overwrite=True)
    print(f"Saved as FIF: {output_path_fif}")

# save metadata as numpy
for i, r in enumerate(raw):

    metadata = {
        'ch_names': r.ch_names,
        'sfreq': r.info['sfreq'],
        'n_channels': len(r.ch_names),
        'n_times': r.n_times
    }
    metadata_path = os.path.join(processed_dir, f'resting_state_metadata{i}.npy')
    np.save(metadata_path, metadata)
    print(f"Saved metadata: {metadata_path}")